In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from torch.utils.data import Dataset
from torchvision.models import efficientnet_b4
from torch.cuda.amp import autocast, GradScaler
from PIL import Image
import pandas as pd
import os
import matplotlib.pyplot as plt

In [2]:


# Define device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# Parameters
img_width, img_height = 224, 224  # EfficientNet-B4 input size
batch_size = 32
num_classes = 5  # Assuming 5 classes for diabetic retinopathy
epochs = 50
clahe_train_dir = r"G:\dataset\diabetic-retinopathy-detection\clahe_train"
test_dir = r"G:\dataset\diabetic-retinopathy-detection\test1"
trainLabels_csv = r"G:\dataset\diabetic-retinopathy-detection\clahe_train\trainLabels_cleaned.csv"
learning_rate = 0.001

# Load labels
labels_df = pd.read_csv(trainLabels_csv)
labels_df['image'] = labels_df['image'] + '.jpeg'  # Ensure the filenames match
labels_df['level'] = labels_df['level'].astype(int)  # Convert labels to integers

# Custom Dataset Class
class RetinopathyDataset(Dataset):
    def __init__(self, labels_df, img_dir, transform=None):
        self.labels_df = labels_df
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.labels_df)

    def __getitem__(self, idx):
        img_name = os.path.join(self.img_dir, self.labels_df.iloc[idx, 0])
        try:
            image = Image.open(img_name).convert('RGB')
        except FileNotFoundError:
            print(f"File {img_name} not found. Skipping...")
            return None, None  # Skip this sample if the image is missing

        label = self.labels_df.iloc[idx, 1]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

# Data augmentation and normalization for training, validation, and test
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop((img_width, img_height), scale=(0.8, 1.0)),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((img_width, img_height)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


cuda


In [3]:
# Split dataset into training and validation
train_size = int(0.8 * len(labels_df))
val_size = len(labels_df) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(
    RetinopathyDataset(labels_df, clahe_train_dir, transform=data_transforms['train']),
    [train_size, val_size]
)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# DataLoader for Test Data
test_dataset = RetinopathyDataset(labels_df, test_dir, transform=data_transforms['val'])
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Load pre-trained EfficientNet model
model = efficientnet_b4(pretrained=True)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
model = model.to(device)

# Mixed Precision Training Setup
scaler = GradScaler()

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

# Learning Rate Scheduler
scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.01, steps_per_epoch=len(train_loader), epochs=epochs)

best_val_loss = float('inf')
patience = 3
trigger_times = 0

train_acc_list, val_acc_list = [], []
train_loss_list, val_loss_list = [], []

g:\anaconda\envs\torch\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
g:\anaconda\envs\torch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B4_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B4_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\Users\Deku2\AppData\Local\Temp\ipykernel_9284\1312063862.py:22: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


In [4]:
# Training Loop
accumulation_steps = 2

for epoch in range(epochs):
    model.train()
    running_loss, correct, total = 0.0, 0, 0

    for i, (inputs, labels) in enumerate(train_loader):
        if inputs is None or labels is None:
            continue  # Skip batch if inputs or labels are missing
        
        inputs, labels = inputs.to(device), labels.to(device)
        labels = labels.long()

        # Zero gradients every accumulation step
        if i % accumulation_steps == 0:
            optimizer.zero_grad()

        # Mixed Precision Training
        with autocast():
            outputs = model(inputs)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()

        # Update weights only after accumulation_steps iterations
        if (i + 1) % accumulation_steps == 0 or (i + 1) == len(train_loader):
            scaler.step(optimizer)
            scaler.update()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    # Calculate training loss and accuracy
    train_loss = running_loss / len(train_loader)
    train_acc = correct / total
    train_loss_list.append(train_loss)
    train_acc_list.append(train_acc)

    # Validation loop
    model.eval()
    running_loss, correct, total = 0.0, 0, 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            labels = labels.long()

            with autocast():
                outputs = model(inputs)
                loss = criterion(outputs, labels)

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Calculate validation loss and accuracy
    val_loss = running_loss / len(val_loader)
    val_acc = correct / total
    val_loss_list.append(val_loss)
    val_acc_list.append(val_acc)

    print(f'Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, '
          f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')

    # Clear the GPU cache after each epoch
    torch.cuda.empty_cache()

    # Early stopping or saving the model logic
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        trigger_times = 0
        torch.save(model.state_dict(), 'best_model3.pth')
    else:
        trigger_times += 1
        if trigger_times >= patience:
            print('Early stopping!')
            break


C:\Users\Deku2\AppData\Local\Temp\ipykernel_24456\463844629.py:20: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
C:\Users\Deku2\AppData\Local\Temp\ipykernel_24456\463844629.py:50: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Epoch 1/50, Train Loss: 0.9571, Train Acc: 0.6494, Val Loss: 0.8611, Val Acc: 0.6856
Epoch 2/50, Train Loss: 0.8268, Train Acc: 0.6966, Val Loss: 0.8290, Val Acc: 0.6970


KeyboardInterrupt: 

In [10]:
# Load the best saved model
model.load_state_dict(torch.load('best_model3.pth'))
model.eval()

# Test accuracy (assuming you have a separate test_loader or can reuse val_loader)
correct, total = 0, 0
with torch.no_grad():
    for inputs, labels in val_loader:  # You can use test_loader if available
        inputs, labels = inputs.to(device), labels.to(device)

        # Ensure labels are of type torch.long
        labels = labels.long()

        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_acc = correct / total
print(f'Test Accuracy: {test_acc:.4f}')


C:\Users\Deku2\AppData\Local\Temp\ipykernel_38792\2610896026.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model3.pth'))


Test Accuracy: 0.7173


In [11]:
# Load the best saved model
model.load_state_dict(torch.load('best_model.pth'))
model.eval()

# Test accuracy (assuming you have a separate test_loader or can reuse val_loader)
correct, total = 0, 0
with torch.no_grad():
    for inputs, labels in val_loader:  # You can use test_loader if available
        inputs, labels = inputs.to(device), labels.to(device)

        # Ensure labels are of type torch.long
        labels = labels.long()

        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_acc = correct / total
print(f'Test Accuracy: {test_acc:.4f}')


C:\Users\Deku2\AppData\Local\Temp\ipykernel_38792\3930918186.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pth'))


Test Accuracy: 0.7077


In [5]:
pip install scikit-learn

  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ------------------------------ --------- 8.4/11.0 MB 47.2 MB/s eta 0:00:01
   ---------------------------------------  11.0/11.0 MB 49.1 MB/s eta 0:00:01
   ---------------------------------------  11.0/11.0 MB 49.1 MB/s eta 0:00:01
   ---------------------------------------- 11.0/11.0 MB 15.3 MB/s eta 0:00:00
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)
Note: you may need to restart the kernel to use updated packages.


In [8]:
from sklearn.metrics import f1_score

# Load the best saved model
model.load_state_dict(torch.load(r'G:\project\webapp_ddr\save_model\best_model3.pth'))
model.eval()

# Initialize lists to store true labels and predictions
all_labels = []
all_preds = []

# Test accuracy and F1-score calculation
correct, total = 0, 0
with torch.no_grad():
    for inputs, labels in val_loader:  # You can use test_loader if available
        inputs, labels = inputs.to(device), labels.to(device)

        # Ensure labels are of type torch.long
        labels = labels.long()

        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Store true labels and predictions for F1-score calculation
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())

# Calculate accuracy
test_acc = correct / total
print(f'Test Accuracy: {test_acc:.4f}')

# Calculate F1-score (average='weighted' for multi-class classification)
f1 = f1_score(all_labels, all_preds, average='weighted')
print(f'F1-Score: {f1:.4f}')


C:\Users\Deku2\AppData\Local\Temp\ipykernel_9284\2514521524.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(r'G:\project\webapp_ddr\save

Test Accuracy: 0.7141
F1-Score: 0.6658


In [9]:
from sklearn.metrics import f1_score

# Load the best saved model
model.load_state_dict(torch.load(r'G:\project\webapp_ddr\save_model\best_model.pth'))
model.eval()

# Initialize lists to store true labels and predictions
all_labels = []
all_preds = []

# Test accuracy and F1-score calculation
correct, total = 0, 0
with torch.no_grad():
    for inputs, labels in val_loader:  # You can use test_loader if available
        inputs, labels = inputs.to(device), labels.to(device)

        # Ensure labels are of type torch.long
        labels = labels.long()

        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Store true labels and predictions for F1-score calculation
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())

# Calculate accuracy
test_acc = correct / total
print(f'Test Accuracy: {test_acc:.4f}')

# Calculate F1-score (average='weighted' for multi-class classification)
f1 = f1_score(all_labels, all_preds, average='weighted')
print(f'F1-Score: {f1:.4f}')


C:\Users\Deku2\AppData\Local\Temp\ipykernel_9284\1859967916.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(r'G:\project\webapp_ddr\save

Test Accuracy: 0.7059
F1-Score: 0.6505


In [11]:
pip install seaborn

Note: you may need to restart the kernel to use updated packages.


In [12]:
from sklearn.metrics import f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Load the best saved model
model.load_state_dict(torch.load(r'G:\project\webapp_ddr\save_model\best_model3.pth'))
model.eval()

# Initialize lists to store true labels and predictions
all_labels = []
all_preds = []

# Test accuracy and F1-score calculation
correct, total = 0, 0
with torch.no_grad():
    for inputs, labels in val_loader:  # You can use test_loader if available
        inputs, labels = inputs.to(device), labels.to(device)

        # Ensure labels are of type torch.long
        labels = labels.long()

        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Store true labels and predictions for F1-score calculation
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())

# Calculate accuracy
test_acc = correct / total
print(f'Test Accuracy: {test_acc:.4f}')

# Calculate F1-score (average='weighted' for multi-class classification)
f1 = f1_score(all_labels, all_preds, average='weighted')
print(f'F1-Score: {f1:.4f}')

# Compute confusion matrix
conf_matrix = confusion_matrix(all_labels, all_preds)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=categories, yticklabels=categories)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()


C:\Users\Deku2\AppData\Local\Temp\ipykernel_9284\2559376308.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(r'G:\project\webapp_ddr\save

KeyboardInterrupt: 